In [1]:
import cv2
import keras
import scipy.io as sio
import h5py

Using TensorFlow backend.


In [2]:
import numpy as np

In [3]:
class DigitStructure:
    def __init__(self):
        self.boxList = None
        self.name = None
        
def load_datafile(filepath):
    
    dataSetfile = h5py.File(filepath,'r')
    return dataSetfile
    
def readInt(intData, dataFile):
    
    intRef = intData[0]
    isReference = isinstance(intRef, h5py.Reference)
    intVal = 0
    if isReference:
        intObj = dataFile[intRef]
        intVal = int(intObj[0])
    else: # Assuming value type
        intVal = int(intRef)
    
    return intVal

def yieldNextInt(intDataset, dataFile):
    
    for intData in intDataset:
        intVal = readInt(intData, dataFile)
        yield intVal 
        
def readDigitStruct(datafile):
    return datafile["digitStruct"]


def get_BBox_attributes(image_box, datafile):
    
    for array in image_box:
        bbList = []
        #take attributes of each digit
        bboxGroupRef = array[0]
        bboxGroup = datafile[bboxGroupRef]
        labelGroup = bboxGroup["label"]
        leftPositionGroup = bboxGroup["left"]
        topPositionGroup = bboxGroup["top"]
        boxWidthGroup = bboxGroup["width"]
        boxHeightGroup = bboxGroup["height"]
        
        left = yieldNextInt(leftPositionGroup, datafile)
        top = yieldNextInt(topPositionGroup, datafile)
        width = yieldNextInt(boxWidthGroup, datafile)
        height = yieldNextInt(boxHeightGroup, datafile)
        
        
        for image_label in yieldNextInt(labelGroup, datafile):
            bbox = BBox()
            bbox.box_label = image_label
            bbox.left = next(left)
            bbox.top = next(top)
            bbox.width = next(width)
            bbox.height = next(height)
            
            bbList.append(bbox)
            
        yield bbList
        
def getNextFileName(nameGroup, datafile):
    
    for array in nameGroup:
        ref = array[0]
        strObj = datafile[ref]
        image_fileName = ''.join(chr(i) for i in strObj)
        yield image_fileName
        


def read_digit_structure(datafile):
    # dictionary of two keys; refs and digitStruct
    print ("""datafile""")
    digitStruct = readDigitStruct(datafile)
    #digitStruct consists of two elements; bbox and name
    
    image_name = digitStruct["name"]
    image_box = digitStruct["bbox"]
    
    image_boxAttributes = get_BBox_attributes(image_box, datafile)
    
    for file_name in getNextFileName(image_name, datafile):
        bbList = next(image_boxAttributes)
        digitStructure = DigitStructure()
        digitStructure.name = file_name
        digitStructure.boxList = bbList
        
        yield digitStructure
        
def access_datafile():
    loadfile = load_datafile('./train/digitStruct.mat')
    testCounter = 0
    for obj in read_digit_structure(loadfile):
        testCounter += 1
        print(obj.name)
        for bbox in obj.boxList:
            print("    {}:{},{},{},{}".format(
                bbox.box_label, bbox.left, bbox.top, bbox.width, bbox.height))
        if testCounter >= 5:
            break
    

In [4]:
class BBox:
    def __init__(self):
        self.label = ""
        self.left = 0
        self.top = 0
        self.width = 0
        self.height = 0

In [5]:
access_datafile()

datafile
1.png
    1:246,77,81,219
    9:323,81,96,219
2.png
    2:77,29,23,32
    3:98,25,26,32
3.png
    2:17,5,8,15
    5:25,5,9,15
4.png
    9:57,13,15,34
    3:72,13,13,34
5.png
    3:52,7,21,46
    1:74,10,15,46


In [65]:
data = h5py.File('./train/digitStruct.mat')

# for item in data['/digitStruct/bbox']:
#     data[item[0]].visititems(print_attrs)

arrays = {}
for k,v in data.items():
    arrays[k] = np.array(v)

print ((arrays["digitStruct"][0]))
print ("")
# digitStruct(300).bbox(2)
print (arrays)
# print (arrays["digitStruct"])

<class 'numpy.str_'>

{'#refs#': array(['#a', '#b', '#c', ..., 'zzx', 'zzy', 'zzz'], 
      dtype='<U3'), 'digitStruct': array(['bbox', 'name'], 
      dtype='<U4')}


In [42]:
a = dict()
a["di"] = [1,3,4]
a["bi"] = [4,5,5]
len(a["di"])

b = np.array([[1,2],[3,5]])
len(b)

2